In [2]:
import urllib
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="ratings_train.txt")
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", filename="ratings_test.txt")

('ratings_test.txt', <http.client.HTTPMessage at 0x2f30421d7b0>)

In [53]:
from nltk.tokenize import word_tokenize
import nltk

from konlpy.tag import Okt
okt = Okt()

In [54]:
def load_data(filename):
    train = []
    with open(filename, 'r', encoding='utf-8') as f:
        count = 0
        for line in f.readlines():
            if count == 1000: break
            count += 1

            line = line.strip()
            id, document, label = line.split('\t')
            if label == '1': label = 'pos'
            else: label = 'neg'    
            train_tup = (document, label)
            train.append(train_tup)
    return train

In [55]:
def pos_tokenize(raw_sent):
    pos_sent = []
    
    sent = okt.pos(raw_sent, norm=True, stem=True)
    for tup in sent:
        word, tag = tup
        word_tag = word + '/' + tag
        pos_sent.append(word_tag)
       
    return ' '.join(pos_sent) 

In [56]:
def make_word_dict(train, use_morph=False):
    all_words = set()
    for tup in train:
        sent, label = tup
        if use_morph: sent = pos_tokenize(sent)
        word_list = word_tokenize(sent)
        for word in word_list:
            all_words.add(word)
            
    return all_words

In [57]:
def make_train_feats(train, all_words, use_morph=False):
    train_features = []
    for tup in train:
        sent, label = tup # sent:'I like you' label: 'pos'
        if use_morph: sent = pos_tokenize(sent)
        word_list = word_tokenize(sent) # ['I', like, you]
        tmp = {set_word: set_word in word_list for set_word in all_words}
        train_feature = (tmp, label)
        train_features.append(train_feature)
        
    return train_features

# Training

In [58]:
use_morph = True

In [59]:
train = load_data('ratings_train.txt')[1:]

In [60]:
all_words = make_word_dict(train, use_morph)

In [61]:
train_features = make_train_feats(train, all_words, use_morph)

In [62]:
classifier = nltk.NaiveBayesClassifier.train(train_features)

In [63]:
classifier.show_most_informative_features(n=10)

Most Informative Features
                쓰레기/Noun = True              neg : pos    =     11.9 : 1.0
                 인생/Noun = True              pos : neg    =     10.0 : 1.0
                 최고/Noun = True              pos : neg    =      9.5 : 1.0
           괜찮다/Adjective = True              pos : neg    =      8.6 : 1.0
                       ; = True              neg : pos    =      8.3 : 1.0
          재미없다/Adjective = True              neg : pos    =      8.1 : 1.0
       ㅡㅡ/KoreanParticle = True              neg : pos    =      8.1 : 1.0
           아깝다/Adjective = True              neg : pos    =      7.6 : 1.0
          지루하다/Adjective = True              neg : pos    =      7.5 : 1.0
           재밌다/Adjective = True              pos : neg    =      7.5 : 1.0


# Test

In [64]:
test = load_data('ratings_test.txt')[1:]

In [65]:
test_features = make_train_feats(test, all_words, use_morph)

In [66]:
classifier.classify(test_features[0][0])

'pos'

# 정확도 계산하기

## use_morph = True (count = 500)

In [67]:
nltk.classify.accuracy(classifier, test_features)

0.7747747747747747

## use_morph = True (count = 1000)

In [68]:
nltk.classify.accuracy(classifier, test_features)

0.7747747747747747

## use_morph = False (count = 500)

In [69]:
nltk.classify.accuracy(classifier, test_features)

0.7747747747747747

## use_morph = False (count = 1000)

In [70]:
nltk.classify.accuracy(classifier, test_features)

0.7747747747747747

# MyNaiveBayesClassifier

In [71]:
class MyNaiveBayesClassifier:
    def __init__(self, k=0.5):
        self.k = k
        self.word_probs = []

In [72]:
classifier = MyNaiveBayesClassifier()

In [73]:
classifier.k

0.5

In [74]:
class MyNaiveBayesClassifier:
    def __init__(self, k=0.5, use_morph=False):
        self.k = k
        self.word_probs = []
        self.use_morph = use_morph
        
        if self.use_morph:
            from konlpy.tag import Okt
            okt = Okt()

In [75]:
classifier = MyNaiveBayesClassifier()

In [76]:
classifier.k

0.5

In [77]:
import math

class MyNaiveBayesClassifier:
    def __init__(self, k=0.5, use_morph=False):
        self.k = k
        self.word_probs = []
        self.use_morph = use_morph
        
        if self.use_morph:
            from konlpy.tag import Okt
            self.okt = Okt()
            
    def load_data(self, file_path):
        docs, labels = [], []
        
        count = 0
        with open(file_path, 'r', encoding='utf-8') as f:
            for line in f.readlines():
                if count == 500: break
                line = line.strip()
                id, doc, label = line.split('\t')
                docs.append(doc)
                if label == '1': label = 'pos'
                else: label = 'neg'
                labels.append(label)
                count += 1
        
        return docs[1:], labels[1:]
    
    def tokenize(self, sentence):
        if self.use_morph:
            pos_sent = []

            sent = self.okt.pos(sentence, norm=True, stem=True)
            for tup in sent:
                word, tag = tup
                word_tag = word + '/' + tag
                pos_sent.append(word_tag)

            sentence = ' '.join(pos_sent) 
        
        return sentence.split()
    
    def count_words(self, docs, labels):
        count_dict = dict()
        for doc, label in zip(docs, labels):
            for word in self.tokenize(doc):
                if word not in count_dict:
                    count_dict[word] = {'pos':0, 'neg':0}
                count_dict[word][label] += 1
                
        print('num of words...', len(count_dict))
        
        return count_dict
    
    def word_prob(self, count_dict, pos_class_num, neg_class_num, k):
        word_prob_list = []
        
        for word in count_dict:
            pos_word_num = count_dict[word]['pos']
            neg_word_num = count_dict[word]['neg']
            
            pos_class_prob = (pos_word_num + k) / (pos_class_num + 2*k)
            neg_class_prob = (neg_word_num + k) / (neg_class_num + 2*k)
        
            tup = (word, pos_class_prob, neg_class_prob)
            word_prob_list.append(tup)
            
        return word_prob_list
    
    def class_prob(self, word_prob_list, test_sentence, use_unseen=False):
        test_words = self.tokenize(test_sentence)
        
        sent_log_pos_class_prob, sent_log_neg_class_prob = 0, 0
        
        for word, word_pos_class_prob, word_neg_class_prob in word_prob_list:
            if word in test_words:
                sent_log_pos_class_prob += math.log(word_pos_class_prob)
                sent_log_neg_class_prob += math.log(word_neg_class_prob)
            else:
                if use_unseen:
                    sent_log_pos_class_prob += math.log(1-word_pos_class_prob)
                    sent_log_neg_class_prob += math.log(1-word_neg_class_prob)
        
        sent_pos_class_prob = math.exp(sent_log_pos_class_prob)
        sent_neg_class_prob = math.exp(sent_log_neg_class_prob)
        
        pos_class_prob = sent_pos_class_prob / (sent_pos_class_prob+sent_neg_class_prob)
        neg_class_prob = sent_neg_class_prob / (sent_pos_class_prob+sent_neg_class_prob)
        
        return pos_class_prob, neg_class_prob 
    
    def train(self, train_file_path):
        train_docs, train_labels = self.load_data(train_file_path)
        
        word_count_dict = self.count_words(train_docs, train_labels)
        
        pos_class_num = 0
        neg_class_num = 0
        for label in train_labels:
            if label == 'pos':
                pos_class_num += 1
            else: 
                neg_class_num += 1
                
        
        self.word_probs = self.word_prob(word_count_dict, pos_class_num, neg_class_num, self.k)
        
    def classify(self, doc, use_unseen=False):
        pos_class_prob, neg_class_prob = self.class_prob(self.word_probs, doc, use_unseen)
        
        if pos_class_prob > neg_class_prob:
            print('pos', pos_class_prob)
        else:
            print('neg', neg_class_prob)
    
    def accuracy(self,file_path,use_unseen=False):
        test_docs, test_labels = self.load_data(file_path)
        acc = 0
        for doc, label in zip(test_docs,test_labels):
            pos_class_prob, neg_class_prob = self.class_prob(self.word_probs, doc, use_unseen)
            if label =='pos':
                if pos_class_prob > neg_class_prob: acc += 1
            elif label =='neg':
                if pos_class_prob < neg_class_prob: acc += 1
        return acc/len(test_labels)

In [78]:
classifier = MyNaiveBayesClassifier()

In [79]:
classifier.train('ratings_train.txt')

num of words... 3055


In [80]:
classifier.classify('최고의 영화예요!', use_unseen=False)

pos 0.923935389133627


In [81]:
classifier.classify('최고의 영화예요!')

pos 0.923935389133627


In [82]:
classifier.classify('최고의 영화예요!', use_unseen=True)

pos 0.9617974172999348


In [83]:
test_docs, test_labels = classifier.load_data('ratings_test.txt')

In [84]:
test_docs[0]

'굳 ㅋ'

In [85]:
classifier.classify(test_docs[0], use_unseen=True)

pos 0.6541187739929468


# 정확도 계산하기

In [86]:
classifier.accuracy('ratings_test.txt', use_unseen=False)

0.5070140280561122

In [87]:
classifier.accuracy('ratings_test.txt', use_unseen=True)

0.6212424849699398

In [88]:
classifier = MyNaiveBayesClassifier(use_morph = True)
classifier.train('ratings_train.txt')

num of words... 2344


In [89]:
classifier.accuracy('ratings_test.txt', use_unseen=False)

0.7214428857715431

In [90]:
classifier.accuracy('ratings_test.txt', use_unseen=True)

0.7154308617234469